In [ ]:
from IPython.display import display_javascript, Javascript, display
from ldap3 import Server, Connection, ALL
import ipywidgets as wg
import os

In [ ]:
def go_to_url(url):
    redirect = 'window.location.assign("{}");'.format(url)
    display_javascript(Javascript(redirect))

In [ ]:
def centered_button(descrip, style, icon, button_width_percent):
    button = wg.Button(description=descrip,
                       layout=wg.Layout(width='auto', grid_area='button'),
                       button_style=style,
                       icon=icon,
                      )
    return wg.GridBox(children=[button],
               layout=wg.Layout(
                   width='100%',
                   grid_template_rows='auto',
                   grid_template_columns='{}% {}% {}%'.format(
                       50 - button_width_percent/2.0,
                       button_width_percent,
                       50 - button_width_percent/2.0,
                   ),
                   grid_template_areas='''
                   ". button ."
                   ''')
              )

# Welcome to Datalab apps
---

In [ ]:
user = os.environ['JUPYTERHUB_USER']
apps_url = 'http://jupyterapp.msedatalab.pl.eso.org/user/{}/apps/nb_apps'.format(user)
ldap_dir = 'ldap://msedatalab'
ldap_base_dn = 'dc=pl,dc=eso,dc=org'
ldap_admin = 'cn=admin'
ldap_admin_passwd = 'admin'
user_menu = []

pass_button = centered_button("Change your Password", "primary", "key", 34)
pass_button.children[0].on_click(
    lambda x: go_to_url('{}/change_pass.ipynb'.format(apps_url))
)
user_menu.append(pass_button)

server = Server(ldap_dir, get_info=ALL)
conn = Connection(server, ldap_admin+','+ldap_base_dn, ldap_admin_passwd, auto_bind=True)

conn.search(ldap_base_dn, '(&(objectclass=PosixGroup)'
                          '(|(cn=qowAdmin)(cn=qowAuditor)(cn=qowWpm)(cn=qowQc))'
                          '(memberUid={}))'.format(user))
if conn.entries != []:
    qow_button = centered_button("Quality of Work", "primary", "clipboard", 34)
    qow_button.children[0].on_click(
        lambda x: go_to_url('{}/qow/user_menu.ipynb'.format(apps_url))
    )
    user_menu.append(qow_button)
    
sao_button = centered_button("SAO Performance Monitoring", "primary", "stethoscope", 34)
sao_button.children[0].on_click(
    lambda x: go_to_url('{}/sao_perfmon/user_menu.ipynb'.format(apps_url))
)
user_menu.append(sao_button)

wg.VBox(user_menu)